<h1>Building Dataset From Riot API</h1>

In [5]:
# teamId == 100 is blue

In [3]:
import json
import requests
import warnings
import time
import pickle

In [4]:
file = open("riot_api.json", 'r')

# Open the credentials file with json.load
credentials = json.load(file)
file.close()

In [3]:
file_path = 'data.pickle'

# with open(file_path, 'wb') as file:
#     pickle.dump(total_match_ids, file)

In [54]:
diamond4_matches = None

with open("diamond4_matches.pickle", 'rb') as file:
    diamond4_matches = pickle.load(file)

In [56]:
print(len(diamond4_matches))

2050


In [8]:
class ApiRequestPipeline:
    """A pipeline class that facilitates chained API calls.

    Uses a series of callback functions defined in a list. 
    Each callback function should return the API request url/s,
    either as a single string or list of strings
    and accept the response of an API request as its parameter
    using the previous callback function's returned url/s.
    Two api rate limits can be added as tuples in the form (request_count, time).
    """

    def __init__(self, callback_list, rate_limit_1=None, rate_limit_2=None, rate_control_type="sprint"):
        self.callback_list = callback_list
        self.short_request_start_time = time.perf_counter()
        self.long_request_start_time = time.perf_counter()
        self.short_request_count = 0
        self.long_request_count = 0
        self.rate_control_type = rate_control_type
        if self.rate_control_type != "sprint" and self.rate_control_type != "walk":
            raise Exception("rate_control_type can only either be 'sprint' or 'walk'")
        
        rate_limit_1 = rate_limit_1 if rate_limit_1 is not None else (0, 0)
        rate_limit_2 = rate_limit_2 if rate_limit_2 is not None else (0, 0)
        
        if rate_limit_1[1] >= rate_limit_2[1]:
            self.long_request_max = rate_limit_1[0]
            self.long_request_cooldown = rate_limit_1[1]
            self.short_request_max = rate_limit_2[0]
            self.short_request_cooldown = rate_limit_2[1]
        else:
            self.long_request_max = rate_limit_2[0]
            self.long_request_cooldown = rate_limit_2[1]
            self.short_request_max = rate_limit_1[0]
            self.short_request_cooldown = rate_limit_1[1]
    
    def __check_requests(self):
        """A helper function that sleeps if it hits the api rate.

        rate_control_type == "sprint": Increments the short request
        and long request counts and then checks if those counts are
        greater than the alloted counts within the specific time period.
        If so, then it sleeps. If not, then it checks if it passed the
        time period without going over the max api limit. If so, then it
        resets the count. Therefore, it assumes that the rate at which it
        will be calling the api is constant, and resets completely instead
        of adhering to a rolling count within the time period.
        
        rate_control_type == "walk": Sleeps enough after every request 
        to always stay under the most restricting of the two rate limits.
        """
        
        # Check for no api limits initialized
        if self.short_request_max == 0 and self.long_request_max == 0:
            return
        if self.rate_control_type == "sprint":
            self.short_request_count += 1
            self.long_request_count += 1
            # If not past long request rate time, and request count is greater than the rate limit max, sleep
            if time.perf_counter() - self.long_request_start_time < self.long_request_cooldown and self.long_request_count >= self.long_request_max:
                print(f"Sleeping for: {self.long_request_cooldown - (time.perf_counter() - self.long_request_start_time):.2f} seconds")
                time.sleep(self.long_request_cooldown - (time.perf_counter() - self.long_request_start_time))
                self.long_request_start_time = time.perf_counter()
                self.short_request_start_time = time.perf_counter()
                self.long_request_count = 1
                self.short_request_count = 1
    
            # If long request was initialized and is past the rate time without going past the rate limit, then reset
            elif self.long_request_cooldown > 0 and time.perf_counter() - self.long_request_start_time > self.long_request_cooldown:
                self.long_request_start_time = time.perf_counter()
                self.long_request_count = 1
    
            # If not past short request rate time, and request count is greater than the rate limit max, sleep
            elif time.perf_counter() - self.short_request_start_time < self.short_request_cooldown and self.short_request_count >= self.short_request_max:
                print(f"Sleeping for: {self.short_request_cooldown - (time.perf_counter() - self.short_request_start_time):.2f} seconds")
                time.sleep(self.short_request_cooldown - (time.perf_counter() - self.short_request_start_time))
                self.short_request_start_time = time.perf_counter()
                self.short_request_count = 1
    
            # If short request was initialized and is past the rate time without going past the rate limit, then reset
            elif self.short_request_cooldown > 0 and time.perf_counter() - self.short_request_start_time > self.short_request_cooldown:
                self.short_request_start_time = time.perf_counter()
                self.short_request_count = 1
        else:
            max_requests_per_second = (self.long_request_max / self.long_request_cooldown) if (self.long_request_max / self.long_request_cooldown) < (self.short_request_max / self.short_request_cooldown) else (self.short_request_max / self.short_request_cooldown)
            time.sleep(1 / max_requests_per_second)
        return

    def make_requests(self, initial_request_url=None):
        """Function to call to start the request loop through the callback function list.

        Can specificy the initial url to request and then start the callback function
        loop. Or you can simply have the first callback function in the list to not 
        take any arguments and simply generates the initial request and returns it. Every 
        callback function then takes the response from the last url returned, or a list of
        reponses if the last function returned a list of urls. Function ultimately returns
        the last response or list of responses obtained from the last callback function
        """
        resp = None
        url = initial_request_url
        if url is not None:
            resp = requests.get(initial_request_url)
        # For each callback function
        for index, f in enumerate(callback_list):
            print(f"Running requests, callback_list index = {index}")
            # Get the callback return
            if resp is not None:
                callback_return = f(resp)
            else:
                callback_return = f()
            # If callback return is a string, then get response treating string as the request url
            if type(callback_return) is str:
                resp = requests.get(callback_return)
                self.__check_requests()
            # If callback return is a list, then create a list of responses
            elif type(callback_return) is list:
                resp = []
                # Loop through list
                for r in callback_return:
                    resp.append(requests.get(r))
                    self.__check_requests()
        return resp

In [9]:
help(ApiRequestPipeline)

Help on class ApiRequestPipeline in module __main__:

class ApiRequestPipeline(builtins.object)
 |  ApiRequestPipeline(callback_list, rate_limit_1=None, rate_limit_2=None, rate_control_type='sprint')
 |
 |  A pipeline class that facilitates chained API calls.
 |
 |  Uses a series of callback functions defined in a list.
 |  Each callback function should return the API request url/s,
 |  either as a single string or list of strings
 |  and accept the response of an API request as its parameter
 |  using the previous callback function's returned url/s.
 |  Two api rate limits can be added as tuples in the form (request_count, time).
 |
 |  Methods defined here:
 |
 |  __init__(self, callback_list, rate_limit_1=None, rate_limit_2=None, rate_control_type='sprint')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  make_requests(self, initial_request_url=None)
 |      Function to call to start the request loop through the callback function list.
 |
 |      Can spe

In [ ]:
def get_summoner_ids():
    return f"https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/BRONZE/III?page=1&api_key={credentials["key"]}"
def get_summoners(resp):
    urls = []
    for summoner in resp.json():
        urls.append(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner["summonerId"]}?api_key={credentials["key"]}")
    return urls
def get_matches(resp_list):
    urls = []
    for resp in resp_list:
        urls.append(f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{resp.json()["puuid"]}/ids?start=0&count=10&api_key={credentials["key"]}")
    return urls

callback_list = [get_summoner_ids, get_summoners, get_matches]

api_pipeline = ApiRequestPipeline(callback_list=callback_list, rate_limit_1=(20, 1), rate_limit_2=(100, 120), rate_control_type="sprint")
match_responses = api_pipeline.make_requests()

In [6]:
ranks = ["iron", "bronze", "silver", "gold", "platinum", "emerald", "diamond"]
subranks = [1, 2, 3, 4]
total_match_ids = {}
for rank in ranks:
    for subrank in subranks:
        with open(f"{rank}{subrank}_matches.pickle", 'rb') as file:
            total_match_ids[f"{rank}{subrank}"] = pickle.load(file)

for key in total_match_ids:
    print(key)
    print(total_match_ids[key])

iron1
['NA1_4909853907', 'NA1_4909829347', 'NA1_4909805539', 'NA1_4908916527', 'NA1_4908038009', 'NA1_4908012916', 'NA1_4907983547', 'NA1_4907982917', 'NA1_4907960760', 'NA1_4904428864', 'NA1_4910504213', 'NA1_4897214176', 'NA1_4892948917', 'NA1_4891912619', 'NA1_4890860557', 'NA1_4890833794', 'NA1_4889092695', 'NA1_4887895971', 'NA1_4887854150', 'NA1_4886861706', 'NA1_4914435558', 'NA1_4914398156', 'NA1_4913840304', 'NA1_4912107693', 'NA1_4907179537', 'NA1_4907092230', 'NA1_4907047819', 'NA1_4907009812', 'NA1_4906039324', 'NA1_4905994471', 'NA1_4914807766', 'NA1_4914779401', 'NA1_4914757131', 'NA1_4914710951', 'NA1_4911077125', 'NA1_4911049957', 'NA1_4910075273', 'NA1_4910045988', 'NA1_4910001931', 'NA1_4909960085', 'NA1_4914478777', 'NA1_4914410481', 'NA1_4914391132', 'NA1_4913474884', 'NA1_4913421193', 'NA1_4912581643', 'NA1_4912553778', 'NA1_4912489944', 'NA1_4912455115', 'NA1_4912428713', 'NA1_4913069452', 'NA1_4913059796', 'NA1_4913043875', 'NA1_4912978308', 'NA1_4912950242', 'NA

In [ ]:
match_data = []
for key in total_match_ids:
    for match_id in total_match_ids[key]:
        resp = requests.get(f"https://americas.api.riotgames.com/lol/match/v5/matches/{match}?api_key={credentials["key"]}")